In [0]:
!git clone https://github.com/matterport/Mask_RCNN.git

In [0]:
!git clone https://github.com/cocodataset/cocoapi

In [0]:
%cd Mask_RCNN

!pip3 install -r requirements.txt
!python3 setup.py install

In [0]:
!wget http://jacarini.dinf.usherbrooke.ca/static/pedestrian%20detection//pedestrian%20detection%20dataset.zip

In [0]:
!unzip "pedestrian detection dataset"


In [0]:
!pip3 install -U scikit-image
!pip3 install -U cython 
!pip3 install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"


In [0]:
# USAGE
# python mask_rcnn.py --mask-rcnn mask-rcnn-coco --image images/example_01.jpg
# python mask_rcnn.py --mask-rcnn mask-rcnn-coco --image images/example_03.jpg --visualize 1

# import the necessary packages
import numpy as np
import argparse
import random
import time
import cv2
import os
import cython
import matplotlib
import sklearn
import tensorflow
import keras
import cv2
import h5py
import imgaug
import IPython
import skimage
import matplotlib.pyplot as plt
import os
import sys




In [0]:
# Root directory of the project
ROOT_DIR =  os.getcwd()  #os.path.abspath("../")

import warnings
warnings.filterwarnings("ignore")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline

In [0]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join('', "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

... done downloading pretrained model!


In [0]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [0]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_coco.hy', config=config)

# Load weights trained on MS-COCO
model.load_weights('mask_rcnn_coco.h5', by_name=True)

In [0]:
#Class names
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [0]:
#Load from the bad weather (blizzard) dataset
!wget http://jacarini.dinf.usherbrooke.ca/static/dataset/badWeather/blizzard.zip
!unzip "blizzard"

In [0]:
#Load from the nighttime dataset
!wget http://jacarini.dinf.usherbrooke.ca/static/dataset/nightVideos.zip
!unzip "nightVideos"

In [0]:
!wget http://jacarini.dinf.usherbrooke.ca/static/dataset/baseline/pedestrians.zip
!unzip 'pedestrians'

In [0]:
# Load and process five frames from the baseline pedestrians dataset
#frames in000417 - in000422
directory = '/content/Mask_RCNN/pedestrian detection dataset/pedestrians/input'
frames = ["in000417.jpg", "in000418.jpg", "in000419.jpg", "in000420.jpg", "in000421.jpg", "in000422.jpg"]
for file in frames:
    image = skimage.io.imread(os.path.join("/content/Mask_RCNN/Mask_RCNN/pedestrians/input", file))
    # Run detection
    results = model.detect([image], verbose=1)
    # Visualize results
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                        class_names, r['scores'])

In [0]:
#Testing under the blizzard environmental condition

directory = '/content/Mask_RCNN/blizzard/input'
frames = ["in001023.jpg", "in001024.jpg", "in001025.jpg", "in001026.jpg", "in001027.jpg", "in001028.jpg"]
for file in frames:
    image = skimage.io.imread(os.path.join("/content/Mask_RCNN/Mask_RCNN/blizzard/input", file))
    # Run detection
    results = model.detect([image], verbose=1)
    # Visualize results
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                        class_names, r['scores'])

In [0]:
#Testing under the nighttime environmental condition

directory = '/content/Mask_RCNN/nightVideos/input'
frames = ["in000004.jpg", "in000005.jpg", "in000006.jpg", "in000007.jpg", "in000008.jpg", "in000011.jpg"]
for file in frames:
    image = skimage.io.imread(os.path.join("/content/Mask_RCNN/Mask_RCNN/nightVideos/fluidHighway/input", file))
    # Run detection
    results = model.detect([image], verbose=1)
    # Visualize results
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                        class_names, r['scores'])

In [0]:
#Quantitative Evaluation
# import the necessary packages
from collections import namedtuple
import numpy as np

import cv2
import numpy as np
import glob
import argparse
import time, datetime
from matplotlib import pyplot as plt
from collections import namedtuple
import random as rng
 
# define the `Detection` object
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])

In [0]:
#Calculating IoU

def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
 
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
 
	# return the intersection over union value
	return iou

In [0]:
#IoU Claculations for the Baseline Data-set
threshold = 100
total_time = datetime.timedelta(seconds=0)
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])
avg_iou = {'blizzard': [], 'pedestrians': [], 'nightVideos': []}
j = 0
tot = 0
#frames 400-699
for subset in avg_iou.keys():

    subset = '/content/Mask_RCNN/' + subset

    print('Processing ' + subset + ' ..')
    
    it = 0
    
    if 'pedestrians' in subset:
        bname = r'/content/Mask_RCNN/Mask_RCNN/pedestrians'

    elif 'blizzard' in subset:
        bname = r'/content/Mask_RCNN/blizzard'
        continue
    else:
        bname = r'/content/Mask_RCNN/nightVideos/fluidHighway'
        continue
    
    iou, sum_iou = 0, 0
    
    glob_min_iou = 100000
    glob_max_iou = -100000

    
    for img, gt in zip(sorted(glob.glob(bname+'/input/in000[4-6]*.jpg')), sorted(glob.glob(bname+'/groundtruth/gt000[4-6]*.png'))):
        
        print(it)
        print(img)
        print(gt)
        if it == 0:

            old_frame = skimage.io.imread(img)

            #old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

            it += 1
            total_time = datetime.timedelta(seconds=0)

        
            
        else:
            # read frame and groundtruth
            
            frame = skimage.io.imread(img)
            gt = cv2.imread(gt)
            #frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # calculate optical flow and run time
            prev_time = time.time()

						

            result = model.detect([frame], verbose=1)

            run_time = datetime.timedelta(seconds=time.time() - prev_time)
            total_time += run_time

            
            k = cv2.waitKey(60) & 0xff
            if k == 27:
                break

            #IOU eval
            print('\tIOU evaluation ..')
            
            #BBoxes generation
            
            
            ret_gt, threshed_img_gt = cv2.threshold(cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY), 100, 255, cv2.THRESH_BINARY)
            
            contours_gt, y = cv2.findContours(threshed_img_gt, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                        
            
            contours_poly_gt = [None]*len(contours_gt)
            boundRect_gt = [None]*len(contours_gt)
            for i, c in enumerate(contours_gt):
                #print(i)
                contours_poly_gt[i] = cv2.approxPolyDP(c, 3, True)
                boundRect_gt[i] = cv2.boundingRect(contours_poly_gt[i])
            
            

            for i in range(len(contours_gt)):
                color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
                cv2.rectangle(gt, (int(boundRect_gt[i][0]), int(boundRect_gt[i][1])), \
                  (int(boundRect_gt[i][0]+boundRect_gt[i][2]), int(boundRect_gt[i][1]+boundRect_gt[i][3])), color, 2)
            
            fig2 = plt.figure(figsize=(25, 25))
            
            
            r = result[0]
            visualize.display_instances(frame, r['rois'], r['masks'], r['class_ids'],
                        class_names, r['scores'])
            y = fig2.add_subplot(1, 1 , 1)
            y.set_title('Contours_gt')
            plt.imshow(gt)
            plt.show()
            
            #IOU calcul
            n = 0
            sum_iou = 0
            if len(contours_gt) != 0:
                max_iou = 0.0
                min_iou = 1.0
                for l in range(r['rois'].shape[0]):
                    for k in range(len(contours_gt)):
                        iou = bb_intersection_over_union((boundRect_gt[k][0], boundRect_gt[k][1], boundRect_gt[k][0]+boundRect_gt[k][2], boundRect_gt[k][1]+boundRect_gt[k][3]), (r['rois'][l][1], r['rois'][l][0], r['rois'][l][3], r['rois'][l][2]))
                        if iou > 0.0:
                            #print(iou)
                            n += 1
                        if iou < min_iou:
                            min_iou = iou
                        if iou > max_iou:
                            max_iou = iou
                        if iou < glob_min_iou:
                            glob_min_iou = iou
                        if iou > glob_max_iou:
                            glob_max_iou = iou
                        sum_iou += iou

                if n >= 0:
                    print('\t\tIOU score for image '+str(it)+': '+str(max_iou))
                    avg_iou[subset.split("/")[3]].append(max_iou)
                    tot += 1
            
            old_gray = frame
            it += 1
    
    print('\tTotal = '+str(tot))
    print('\tAverage IOU score for subset '+subset+' = ' + str(sum(avg_iou[subset.split("/")[3]])/tot) + '\n')
    print('\tMax IOU score for subset '+subset+' = ' + str(glob_max_iou) + '\n')
    print('\tMin IOU score for subset '+subset+' = ' + str(glob_min_iou) + '\n')
    #print('\tAverage runtime for subset '+subset+' = ' + str(total_time/1000) + '\n')
    j += 1
    
cv2.destroyAllWindows()


In [0]:
#IoU calculations for the Blizzard dataset
threshold = 100
total_time = datetime.timedelta(seconds=0)
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])
avg_iou = {'blizzard': [], 'pedestrians': [], 'nightVideos': []}
j = 0
tot = 0
#frames 3400-3699
for subset in avg_iou.keys():

    subset = '/content/Mask_RCNN/' + subset

    print('Processing ' + subset + ' ..')
    
    it = 0
    
    if 'pedestrians' in subset:
        bname = r'/content/Mask_RCNN/Mask_RCNN/pedestrians'
        continue
    elif 'blizzard' in subset:
        bname = r'/content/Mask_RCNN/Mask_RCNN/blizzard'
        
    else:
        bname = r'/content/Mask_RCNN/nightVideos/fluidHighway'
        continue
        
    
    iou, sum_iou = 0, 0
    
    glob_min_iou = 100000
    glob_max_iou = -100000

    
    for img, gt in zip(sorted(glob.glob(bname+'/input/in003[4-6]*.jpg')), sorted(glob.glob(bname+'/groundtruth/gt003[4-6]*.png'))):
        
        print(it)
        print(img)
        print(gt)
        if it == 0:

            old_frame = skimage.io.imread(img)

            #old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

            it += 1
            total_time = datetime.timedelta(seconds=0)

        
            
        else:
            # read frame and groundtruth
            
            frame = skimage.io.imread(img)
            gt = cv2.imread(gt)
            #frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # calculate optical flow and run time
            prev_time = time.time()

						

            result = model.detect([frame], verbose=1)

            run_time = datetime.timedelta(seconds=time.time() - prev_time)
            total_time += run_time

            
            k = cv2.waitKey(60) & 0xff
            if k == 27:
                break

            #IOU eval
            print('\tIOU evaluation ..')
            
            #BBoxes generation
            
            
            ret_gt, threshed_img_gt = cv2.threshold(cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY), 100, 255, cv2.THRESH_BINARY)
            
            contours_gt, y = cv2.findContours(threshed_img_gt, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                        
            
            contours_poly_gt = [None]*len(contours_gt)
            boundRect_gt = [None]*len(contours_gt)
            for i, c in enumerate(contours_gt):
                #print(i)
                contours_poly_gt[i] = cv2.approxPolyDP(c, 3, True)
                boundRect_gt[i] = cv2.boundingRect(contours_poly_gt[i])
            
            

            for i in range(len(contours_gt)):
                color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
                cv2.rectangle(gt, (int(boundRect_gt[i][0]), int(boundRect_gt[i][1])), \
                  (int(boundRect_gt[i][0]+boundRect_gt[i][2]), int(boundRect_gt[i][1]+boundRect_gt[i][3])), color, 2)
            
            fig2 = plt.figure(figsize=(25, 25))
            
            
            r = result[0]
            visualize.display_instances(frame, r['rois'], r['masks'], r['class_ids'],
                        class_names, r['scores'])
            y = fig2.add_subplot(1, 1 , 1)
            y.set_title('Contours_gt')
            plt.imshow(gt)
            plt.show()
            
            #IOU calcul
            n = 0
            sum_iou = 0
            if len(contours_gt) != 0:
                max_iou = 0.0
                min_iou = 1.0
                for l in range(r['rois'].shape[0]):
                    for k in range(len(contours_gt)):
                        iou = bb_intersection_over_union((boundRect_gt[k][0], boundRect_gt[k][1], boundRect_gt[k][0]+boundRect_gt[k][2], boundRect_gt[k][1]+boundRect_gt[k][3]), (r['rois'][l][1], r['rois'][l][0], r['rois'][l][3], r['rois'][l][2]))
                        if iou > 0.0:
                            #print(iou)
                            n += 1
                        if iou < min_iou:
                            min_iou = iou
                        if iou > max_iou:
                            max_iou = iou
                        if iou < glob_min_iou:
                            glob_min_iou = iou
                        if iou > glob_max_iou:
                            glob_max_iou = iou
                        sum_iou += iou

                if n >= 0:
                    print('\t\tIOU score for image '+str(it)+': '+str(max_iou))
                    avg_iou[subset.split("/")[3]].append(max_iou)
                    tot += 1
            
            old_gray = frame
            it += 1
    
    print('\tTotal = '+str(tot))
    print('\tAverage IOU score for subset '+subset+' = ' + str(sum(avg_iou[subset.split("/")[3]])/tot) + '\n')
    print('\tMax IOU score for subset '+subset+' = ' + str(glob_max_iou) + '\n')
    print('\tMin IOU score for subset '+subset+' = ' + str(glob_min_iou) + '\n')
    #print('\tAverage runtime for subset '+subset+' = ' + str(total_time/1000) + '\n')
    j += 1
    
cv2.destroyAllWindows()

In [0]:
#IoU Calculations for the Nighttime Dataset
threshold = 100
total_time = datetime.timedelta(seconds=0)
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])
avg_iou = {'blizzard': [], 'pedestrians': [], 'nightVideos': []}
j = 0
tot = 0
#frames 400-699
for subset in avg_iou.keys():

    subset = '/content/Mask_RCNN/' + subset

    print('Processing ' + subset + ' ..')
    
    it = 0
    
    if 'pedestrians' in subset:
        bname = r'/content/Mask_RCNN/Mask_RCNN/pedestrians'
        continue
    elif 'blizzard' in subset:
        bname = r'/content/Mask_RCNN/blizzard'
        continue
    else:
        bname = r'/content/Mask_RCNN/Mask_RCNN/nightVideos/fluidHighway'
        
    
    iou, sum_iou = 0, 0
    
    glob_min_iou = 100000
    glob_max_iou = -100000

    
    for img, gt in zip(sorted(glob.glob(bname+'/input/in000[4]*.jpg')), sorted(glob.glob(bname+'/groundtruth/gt000[4]*.png'))):
        
        print(it)
        print(img)
        print(gt)
        if it == 0:

            old_frame = skimage.io.imread(img)

            #old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

            it += 1
            total_time = datetime.timedelta(seconds=0)

        
            
        else:
            # read frame and groundtruth
            
            frame = skimage.io.imread(img)
            gt = cv2.imread(gt)
            #frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # calculate optical flow and run time
            prev_time = time.time()

						

            result = model.detect([frame], verbose=1)

            run_time = datetime.timedelta(seconds=time.time() - prev_time)
            total_time += run_time

            
            k = cv2.waitKey(60) & 0xff
            if k == 27:
                break

            #IOU eval
            print('\tIOU evaluation ..')
            
            #BBoxes generation
            
            
            ret_gt, threshed_img_gt = cv2.threshold(cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY), 100, 255, cv2.THRESH_BINARY)
            
            contours_gt, y = cv2.findContours(threshed_img_gt, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                        
            
            contours_poly_gt = [None]*len(contours_gt)
            boundRect_gt = [None]*len(contours_gt)
            for i, c in enumerate(contours_gt):
                #print(i)
                contours_poly_gt[i] = cv2.approxPolyDP(c, 3, True)
                boundRect_gt[i] = cv2.boundingRect(contours_poly_gt[i])
            
            

            for i in range(len(contours_gt)):
                color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
                cv2.rectangle(gt, (int(boundRect_gt[i][0]), int(boundRect_gt[i][1])), \
                  (int(boundRect_gt[i][0]+boundRect_gt[i][2]), int(boundRect_gt[i][1]+boundRect_gt[i][3])), color, 2)
            
            fig2 = plt.figure(figsize=(25, 25))
            
            
            r = result[0]
            visualize.display_instances(frame, r['rois'], r['masks'], r['class_ids'],
                        class_names, r['scores'])
            y = fig2.add_subplot(1, 1 , 1)
            y.set_title('Contours_gt')
            plt.imshow(gt)
            plt.show()
            
            #IOU calcul
            n = 0
            sum_iou = 0
            if len(contours_gt) != 0:
                max_iou = 0.0
                min_iou = 1.0
                for l in range(r['rois'].shape[0]):
                    for k in range(len(contours_gt)):
                        iou = bb_intersection_over_union((boundRect_gt[k][0], boundRect_gt[k][1], boundRect_gt[k][0]+boundRect_gt[k][2], boundRect_gt[k][1]+boundRect_gt[k][3]), (r['rois'][l][1], r['rois'][l][0], r['rois'][l][3], r['rois'][l][2]))
                        if iou > 0.0:
                            #print(iou)
                            n += 1
                        if iou < min_iou:
                            min_iou = iou
                        if iou > max_iou:
                            max_iou = iou
                        if iou < glob_min_iou:
                            glob_min_iou = iou
                        if iou > glob_max_iou:
                            glob_max_iou = iou
                        sum_iou += iou

                if n >= 0:
                    print('\t\tIOU score for image '+str(it)+': '+str(max_iou))
                    avg_iou[subset.split("/")[3]].append(max_iou)
                    tot += 1
            
            old_gray = frame
            it += 1
    
    print('\tTotal = '+str(tot))
    print('\tAverage IOU score for subset '+subset+' = ' + str(sum(avg_iou[subset.split("/")[3]])/tot) + '\n')
    print('\tMax IOU score for subset '+subset+' = ' + str(glob_max_iou) + '\n')
    print('\tMin IOU score for subset '+subset+' = ' + str(glob_min_iou) + '\n')
    #print('\tAverage runtime for subset '+subset+' = ' + str(total_time/1000) + '\n')
    j += 1
    
cv2.destroyAllWindows()
